In [3]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import re
import json
import urllib.request

In [33]:
file_path = '/2nd_project/recipe.json'
def toJson(recipe_dict):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(recipe_dict, file, ensure_ascii=False, indent='\t')


def url_func(n,m):
    url = 'https://www.10000recipe.com/recipe/list.html?order=reco&page='
    url_list = []
    
    for num in range(n,m):
        req = urllib.request.Request(url + str(num))
        html = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(html, 'html.parser')
        
        try:
            res = soup.find(class_='common_sp_list_ul ea4')
            for i in res.find_all('a'):
                url_tmp = i.get('href')
                url_list.append(url_tmp)
        except(AttributeError):
            pass
    return url_list

In [ ]:
num_id = 0
food_lists = []
url_lists = url_func(1,2)

for url_str in url_lists:
    url = 'https://www.10000recipe.com'
    url = url + url_str
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    #menu name
    res = soup.find('div', 'view2_summary')
    res = res.find('h3')
    menu_name = res.get_text().replace('\n',' ').replace('"','')

    #menu image
    res = soup.find('div', 'centeredcrop')
    res = res.find('img')
    menu_img = res.get('src')

    # ingredients
    res = soup.find('div','ready_ingre3')
    ingre_dict = {}
    ingre_list = []
    try:
        for n in res.find_all('ul'):
            for tmp in n.find_all('li'):
                ingredient_name = tmp.get_text().replace('\n','').replace(' ','')
                count = tmp.find('span')
                ingredient_tmp = count.get_text()
                ingredient_name = re.sub(ingredient_tmp, '',ingredient_name)
                ingredient_unit = ingredient_tmp.replace('/','').replace('+','')
                ingredient_unit = ''.join([i for i in ingredient_unit if not i.isdigit()])
                ingredient_count = re.sub(ingredient_unit, '', ingredient_tmp)

                ingre_dict = {"ingre_name": ingredient_name,
                             "ingre_count": ingredient_count,
                             "ingre_unit": ingredient_unit}

                ingre_list.append(ingre_dict)
    except(AttributeError):
        pass

    # recipe
    recipe_list = []
    res = soup.find('div','view_step')
    ind = 1
    for n in res.find_all('div', 'view_step_cont'):
        recipe_step_txt = n.get_text().replace('\n',' ')
        recipe_list.append(str(ind) + '. ' + recipe_step_txt)
        ind += 1

    food_dict = {"id":num_id,
                "name":menu_name,
                "img":menu_img,
                "ingre":ingre_list,
                "reci":recipe_list}
    num_id += 1
    food_lists.append(food_dict)
#     print(food_lists)

toJson(food_lists)

In [31]:
url = 'https://www.10000recipe.com/recipe/6854292'
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
    
#menu name
res = soup.find('div', 'view2_summary')
res = res.find('h3')
menu_name = res.get_text().replace('\n',' ').replace('"','')

#menu image
res = soup.find('div', 'centeredcrop')
res = res.find('img')
menu_img = res.get('src')

# ingredients
res = soup.find('div','ready_ingre3')
ingre_dict = {}
ingre_list = []
try:
    for n in res.find_all('ul'):
        for tmp in n.find_all('li'):
            ingredient_name = tmp.get_text().replace('\n','').replace(' ','')
            count = tmp.find('span')
            ingredient_tmp = count.get_text()
            ingredient_name = re.sub(ingredient_tmp, '',ingredient_name)
            ingre_list.append(ingredient_name)
except(AttributeError):
    pass

# 요리시간
res = soup.find('span', 'view2_summary_info2')
time = res.get_text().replace('이내','').replace('분','').replace(' ','')

# 조리 방법+이미지
recipe_list = []
recipe_dict = []
recipe_imges = []
res = soup.find('div','view_step')
ind = 1
for n in res.find_all('div', 'view_step_cont', 'media-body'):
    recipe_step_txt = n.get_text().replace('\n',' ')
    recipe_dict = {"txt":recipe_step_txt}
    recipe_list.append(str(ind) + '. ' + recipe_step_txt)
    ind += 1
    
for n in res.find_all('div', 'view_step_cont', 'media-right'):
    res = n.find('img')
    recipe_imges.append(res.get('src'))
    
    
food_dict = {"id":0,
            "name":menu_name,
            "thumbnail":menu_img,
            "url":url,
             "ingre_list":ingre_list,
            "time":time,
            "recipe":recipe_list,
            "recipe_img":recipe_imges}
print(food_dict)

{'id': 0, 'name': '떡꼬치 만들기', 'thumbnail': 'https://recipe1.ezmember.co.kr/cache/recipe/2016/08/09/500316fab5981bf25bb405cc660267e91.jpg', 'url': 'https://www.10000recipe.com/recipe/6854292', 'ingre_list': ['고추장', '케찹', '진간장', '설탕', '올리고당', '다진마늘', '통깨', '떡볶이떡'], 'time': '30', 'recipe': ['1. 떡은 물에 행궈서 한번 데쳐줘요~ 그래야 떡이 좀더 야들야들 하거든요~', '2. 데친다음 꺼내서 식혀주시면 되세요~ 여기에 살짝 기름을 둘러서 섞으면 서도 덜 들러붙을 거에요~', '3. 양념비율대로 양념장을 만들어주세요 이건 정말 짜지도 않고 달콤하니 맛나요^^ 아이들이 매운걸 못먹으면 고추장양을 조절하시면 될듯해요', '4. 황금비율로 떡꼬치 소스를 만들었다면 이제 떡을 기름에 구워야겠죠~', '5. 기름을 넉넉히 둘러주세요~ 떡을 기름에 튀기는건 절대 하지마시구요;; 저 예전에 그랬다가...정말 집 날라가는 줄알았어요..ㅠㅠ 아직도 그때 트라우마가...ㅋㅋ그래서 기름 조금만 튀어도 막 놀래자빠짐....', '6. 저는 귀찮아서 다 구운다음에 꽂아줬어요~ 왜냐면..꼬치에 꽂은상태로 구우면 몇개 못하거든요...ㅋㅋㅋ 그래서...전 다한다음 꽂아줍니다..ㅎㅎ', '7. 이렇게 다구워지면 꼬치에 꽂아주면 됩니다^^ 이제 아까 만들었 두었던 떡꼬치 소스만 쓱쓱 바랄주면 되겠죠~ 애들 간식 만들기 참 쉽죵~ㅎ 간식도 종류별로 만드는거 알아두면 좋을텐데...ㅋㅋㅋ 제 용량이 많이 저장이 안되는지라...ㅎㅎ'], 'recipe_img': ['https://recipe1.ezmember.co.kr/cache/recipe/2016/08/09/3fde1193e9c844757609ad66e543b9f01.jpg', 'https://recipe1.ezmember.

In [ ]:
url_lists = url_func(1,3637)
print(url_lists)